In [174]:
import torch
N = 2
chi = 2
d = 2
layers = 1

rand_input = torch.randn(N, 5, 2, dtype=torch.float64)
rand_input /= torch.norm(rand_input, dim=-1, keepdim=True)



In [175]:
from scipy.stats import unitary_group, ortho_group

def random_unitary(chi, dtype=torch.float64, device=torch.device("cpu")):
    """
    Generate a random unitary tensor based on the dtype.

    Args:
        dtype (torch.dtype): The desired data type of the tensor.
        device (torch.device): The device on which to create the tensor.

    Returns:
        torch.Tensor: A randomly initialized unitary tensor of shape (chi^2, chi^2).
    """
    if dtype == torch.float64:
        # Generate a random orthogonal matrix for real-valued tensors
        unitary = torch.from_numpy(ortho_group.rvs(chi ** 2)).to(dtype=dtype, device=device)
    elif dtype == torch.complex128:
        # Generate a random unitary matrix for complex-valued tensors
        unitary = torch.from_numpy(unitary_group.rvs(chi ** 2)).to(dtype=dtype, device=device)
    else:
        raise ValueError("Unsupported dtype for random_unitary. Use torch.float64 or torch.complex128.")
    
    return unitary.reshape(chi, chi, chi, chi)

In [176]:
from importlib import reload
import umps
import tpcp_mps

reload(umps)
reload(tpcp_mps)
# pass
# Generate a random unitary tensor
random_Us = [random_unitary(chi, dtype=torch.float64, device=torch.device("cpu")) for _ in range(N - 1)]
random_Us = torch.stack(random_Us, dim=0)

# # Initialize uMPS with the random unitary
umps_model = umps.uMPS(
    N=N,
    chi=chi,
    d=d,
    l=d,
    layers=layers,
    device=torch.device("cpu"),
    init_with_identity=False
)
# umps_model.initialize_MPS(init_with=random_Us)

tpcpmps_model = tpcp_mps.MPSTPCP(N=N, K=1, d=d, with_identity=False)

# # For now, we'll proceed without initializing tpcpmps_model with the random unitary
# # and focus on ensuring both models produce matching outputs when initialized correctly

# # Get outputs from both models

Path is not set, setting...
Found the path
Initialized MPS params


In [183]:
umps_model.initialize_MPS(init_with=random_Us)
tpcpmps_model.kraus_ops.init_params(init_with=random_Us.reshape(-1, 1, chi**2, chi**2))  # Adjust based on actual method

Initialized MPS params


In [184]:
tpcpmps_output = tpcpmps_model(rand_input.permute(1, 0, 2))
umps_output = umps_model(rand_input)

In [185]:
tpcpmps_output - umps_output

tensor([-0.6662, -0.5324, -0.2442, -0.2577,  0.0340], dtype=torch.float64,
       grad_fn=<SubBackward0>)

In [194]:
list(tpcpmps_model.kraus_ops.parameters())

[Parameter containing:
 tensor([[-0.9290, -0.2097,  0.1586, -0.2606],
         [-0.1455,  0.8681, -0.3036, -0.3647],
         [ 0.3403, -0.2109,  0.2814, -0.8721],
         [ 0.0083,  0.3973,  0.8964,  0.1964]], dtype=torch.float64,
        requires_grad=True)]

In [196]:
rho1

tensor(1., dtype=torch.float64)

In [204]:
U = random_Us[0].reshape(chi**2, chi**2)
# Get first element of input
input1 = rand_input[0, 0, :]
input2 = rand_input[1, 0, :]
rho1 = torch.einsum('i,j->ij', input1, input1.conj())
rho2 = torch.einsum('i,j->ij', input2, input2.conj())
rho = torch.kron(rho1, rho2)
rho = U @ rho @ U.T.conj()


p00 = rho[0, 0]
p01 = rho[1, 1]
p10 = rho[2, 2]
p11 = rho[3, 3]

p0 = p00 + p01
p1 = p00 + p10

In [212]:
p0, p1

(tensor(0.4389, dtype=torch.float64), tensor(0.2103, dtype=torch.float64))

In [213]:
tpcpmps_output

tensor([0.2103, 0.4553, 0.6258, 0.3407, 0.9943], dtype=torch.float64,
       grad_fn=<SelectBackward0>)